In [ ]:
import pandas as pd
import sqlite3
import json

# 1. Load CSV Data
def load_csv_data(file_path):
    """Loads the orders dataset."""
    return pd.read_csv(file_path)

# 2. Load JSON Data
def load_json_data(file_path):
    """Loads the user profile dataset."""
    with open(file_path, 'r') as f:
        data = json.load(f)
    return pd.DataFrame(data)

# 3. Load SQL Data
def load_sql_data(file_path):
    """Parses restaurant details from SQL insert statements."""
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    with open(file_path, 'r') as f:
        sql_script = f.read()
    cursor.executescript(sql_script)
    return pd.read_sql_query("SELECT * FROM restaurants", conn)

# 4. Merge and Pre-process Data
def create_final_dataset(orders, users, restaurants):
    # Merge datasets
    merged = pd.merge(orders, users, on='user_id', how='left')
    final = pd.merge(merged, restaurants, on='restaurant_id', how='left', suffixes=('', '_info'))
    
    # --- PRE-PROCESSING FOR HACKATHON QUESTIONS ---
    
    # Convert date to datetime objects
    final['order_date'] = pd.to_datetime(final['order_date'], dayfirst=True)
    
    # Extract Quarter (Q1, Q2, Q3, Q4)
    final['quarter'] = 'Q' + final['order_date'].dt.quarter.astype(str)
    
    # Create Rating Ranges (Bins)
    bins = [3.0, 3.5, 4.0, 4.5, 5.0]
    labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
    final['rating_range'] = pd.cut(final['rating'], bins=bins, labels=labels, include_lowest=True)
    
    return final

# Main Execution logic
if __name__ == "__main__":
    # Load raw files
    df_orders = load_csv_data('orders.csv')
    df_users = load_json_data('users.json')
    df_restaurants = load_sql_data('restaurants.sql')

    # Create the single source of truth
    final_df = create_final_dataset(df_orders, df_users, df_restaurants)

    # Save for submission
    final_df.to_csv('final_food_delivery_dataset.csv', index=False)
    print("✅ Success: final_food_delivery_dataset.csv has been created.")

    # --- QUICK VERIFICATION FOR HACKATHON ANSWERS ---
    print("\n--- HACKATHON INSIGHTS ---")
    
    # Q: Total revenue by City (Gold Members)
    gold_city_rev = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
    print(f"Top Gold City Revenue:\n{gold_city_rev.idxmax()} - {gold_city_rev.max():.2f}")

    # Q: Highest Revenue Quarter
    q_rev = final_df.groupby('quarter')['total_amount'].sum()
    print(f"Highest Revenue Quarter: {q_rev.idxmax()}")

    # Q: Count of distinct users spending > 1000
    user_spend = final_df.groupby('user_id')['total_amount'].sum()
    print(f"Users spending > 1000: {(user_spend > 1000).sum()}")

Final merged dataset saved as 'final_food_delivery_dataset.csv'
Total Gold Revenue: 3975364.89
